Встановимо модулі matplotlib, numpy, pandas, ipython:

In [1]:
pip install numpy pandas matplotlib ipython

Note: you may need to restart the kernel to use updated packages.


# Завдання 1

Для кожної із адміністративних одиниць України завантажити тестові структуровані файли, що містять значення VHI-індексу. 
Ця процедура має бути автоматизована, параметром процедури має бути індекс (номер) області. 
При зберіганні файлу до його імені потрібно додати дату та час завантаження.

Передбачити повторні запуски скрипту, довантаження нових даних та колізію
даних;

In [2]:
import urllib.request
import hashlib
import os
from datetime import datetime

# Папка для збереження CSV-файлів
DATA_DIR = "CSV_Files" 
if not os.path.exists(DATA_DIR):
    os.makedirs(DATA_DIR)

# Функція для обчислення хешу файлу (SHA-256)
def compute_sha256(file_path):
    """Обчислює SHA-256 хеш файлу для перевірки дублікатів."""
    sha256 = hashlib.sha256()
    with open(file_path, "rb") as f:
        for chunk in iter(lambda: f.read(4096), b""):
            sha256.update(chunk)
    return sha256.hexdigest()

# Функція для перевірки коректності відповіді сервера
def is_valid_response(response):
    """Перевіряє, чи отримані коректні дані (не HTML-сторінка з помилкою)."""
    content_type = response.getheader('Content-Type', '')
    return 'text/plain' in content_type or 'csv' in content_type

# Функція для завантаження VHI-даних для конкретної області
def download_vhi_data(region_ID):
    """Завантажує VHI-дані для області та зберігає у файлі з унікальним ім'ям."""
    
    # Формування URL для отримання даних
    url = f"https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={region_ID}&year1=1981&year2=2024&type=Mean"
    
    # Генерація унікальної назви файлу із зазначенням часу
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_file_path = os.path.join(DATA_DIR, f"VHI_{region_ID}_{timestamp}.csv")

    try:
        # Відправка запиту та отримання даних
        response = urllib.request.urlopen(url)
        content = response.read()

        # Запис отриманих даних у CSV-файл
        with open(output_file_path, 'wb') as file:
            file.write(content)

        # Перевірка наявності дубліката
        for stored_file in os.listdir(DATA_DIR):
            if stored_file.endswith(".csv") and f"Region{region_ID}" in stored_file:
                stored_path = os.path.join(DATA_DIR, stored_file)
                
                # Якщо файл ідентичний, видаляємо новий дублікат
                if stored_path != output_file_path and os.path.exists(stored_path):
                    if compute_sha256(stored_path) == compute_sha256(output_file_path):
                        print(f"Файл для області {region_ID} вже є. Видаляю дубль.")
                        os.remove(output_file_path)
                        return
                        
        print(f"Файл для області {region_ID} успішно завантажено: {output_file_path}")

    except Exception as err:
        print(f"Помилка при отриманні даних для області {region_ID}: {err}")

# Завантаження даних для всіх 27 адміністративних одиниць України
    download_vhi_data(region_ID)

# Завдання 2

Зчитати завантажені текстові файли у фрейм
(https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html) (детальніше
про роботу із фреймами буде розказано у подальших лабораторних роботах).
Імена стовбців фрейму мають бути змістовними та легкими для сприйняття (не
повинно бути спеціалізованих символів, пробілів тощо). Ця задача має бути
реалізована у вигляді окремої процедури, яка на вхід приймає шлях до
директорії, в якій зберігаються файли;

In [3]:
import pandas as pd
import os
import re

# Папка з CSV-файлами та вихідний файл
DATA_DIR = "CSV_Files"  # Моя директорія
output_path = "source_file.csv" # Файл для збереження 

# Імена колонок для зчитування
COLUMN_NAMES = ["Year", "Week", "SMN", "SMT", "VCI", "TCI", "VHI", "PROVINCE_ID"]

# Порожній DataFrame для об'єднання даних
combined_data = pd.DataFrame(columns=COLUMN_NAMES)

# Отримуємо список CSV-файлів, відсортованих за номером області
files = sorted([f for f in os.listdir(DATA_DIR) if f.endswith('.csv')], 
               key=lambda x: int(re.findall(r'\d+', x)[0]))

# Обробка кожного файлу
for file_name in files:
    file_path = os.path.join(DATA_DIR, file_name)

    try:
        # Отримуємо ID області з назви файлу
        province_id = int(re.findall(r'\d+', file_name)[0])

        # Зчитуємо файл, пропускаючи перші 2 рядки
        df = pd.read_csv(file_path, skiprows=2, names=COLUMN_NAMES)

        # Видаляємо HTML-теги з Year
        df["Year"] = df["Year"].astype(str).str.replace(r'<tt><pre>|</pre></tt>', '', regex=True)

        # pd.to_numeric(..., errors='coerce') замінює некоректні значення на NaN, якщо такі є.
        df["Year"] = pd.to_numeric(df["Year"], errors='coerce')
        df["Week"] = pd.to_numeric(df["Week"], errors='coerce')

        # Видаляє рядки з NaN у Year та Week, щоб уникнути помилок при подальших обчисленнях
        df.dropna(subset=["Year", "Week"], inplace=True)

        # Перетворює Year і Week на цілі числа (int):
        df["Year"] = df["Year"].astype(int)
        df["Week"] = df["Week"].astype(int)

        # Додаємо ID області у колонку "PROVINCE_ID"
        df["PROVINCE_ID"] = province_id

        # Видаляємо рядки з некоректними VHI та NaN
        df = df[df["VHI"] != -1].dropna()

        # Додаємо оброблені дані в загальний DataFrame
        combined_data = pd.concat([combined_data, df], ignore_index=True)

    except Exception as e:
        print(f"Помилка при читанні файлу {file_name}: {e}")

# Зберігаємо результат у файл
combined_data.to_csv(output_path, index=False)

# Виводимо перші 10 рядків
print("Об'єднані дані успішно збережено у", output_path)
print("Перші 10 рядків:")
print(combined_data.head(10))
print("Ось останні 10 рядків:")
print(combined_data.tail(10))


Об'єднані дані успішно збережено у source_file.csv
Перші 10 рядків:
Empty DataFrame
Columns: [Year, Week, SMN, SMT, VCI, TCI, VHI, PROVINCE_ID]
Index: []
Ось останні 10 рядків:
Empty DataFrame
Columns: [Year, Week, SMN, SMT, VCI, TCI, VHI, PROVINCE_ID]
Index: []


# Завдання 3

Реалізувати окрему процедуру, яка змінить індекси областей, які використані на
порталі NOAA (за англійською абеткою) на наступні, за українською (виключно
старі індекси на нові):

In [4]:
import pandas as pd

def update_province_ids(df):
    
    province_mapping = {
        1: 24,  2: 26,  3: 25,  4: 27,  5: 3,   6: 4,   7: 8,  
        8: 21,  9: 22, 10: 23, 11: 10, 12: 9,  13: 11, 14: 12, 
       15: 13, 16: 14, 17: 15, 18: 16, 19: 17, 20: 18, 21: 19, 
       22: 20, 23: 6,  24: 1,  25: 2,  26: 7,  27: 5
    }

    # Переконуємося, що колонка "PROVINCE_ID" є у DataFrame
    if "PROVINCE_ID" not in df.columns:
        print("Колонка 'PROVINCE_ID' не знайдена у DataFrame!")
        return df

    # Оновлення значень
    df["PROVINCE_ID"] = df["PROVINCE_ID"].map(province_mapping)

    return df

# Оновлення індексів у `combined_data`
combined_data = update_province_ids(combined_data)

# Збереження у файл
output_file = "Updated_Provinces.csv"
combined_data.to_csv(output_file, index=False)

# Вивід результату
print(f"Оновлений файл збережено: {output_file}")
print(combined_data.head(10))


Оновлений файл збережено: Updated_Provinces.csv
Empty DataFrame
Columns: [Year, Week, SMN, SMT, VCI, TCI, VHI, PROVINCE_ID]
Index: []


# Завдання 4

### Реалізувати процедури для формування вибірок наступного виду (включаючи елементи аналізу):

o Ряд VHI для області за вказаний рік;

1. Отримати ряд VHI для області за вказаний рік

In [5]:
def get_vhi_for_region_year(df, province_id, year, start_week, end_week):

    return df[(df["PROVINCE_ID"] == province_id) & (df["Year"] == year)& (df["Week"]>=start_week)&df["Week"]<=end_week ][["Year", "Week", "VHI"]]

vhi_data = get_vhi_for_region_year(combined_data, province_id=10, year=2020, start_week=15, end_week=20)
print(vhi_data)


Empty DataFrame
Columns: [Year, Week, VHI]
Index: []


2. Пошук екстремумів (min, max), середнього, медіани

In [6]:
def get_vhi_statistics(df, province_ids, years):

    # Фільтруємо дані за обраними областями та роками
    filtered_df = df[(df["PROVINCE_ID"].isin(province_ids)) & (df["Year"].isin(years))]

    # Групуємо за областю та роком, обчислюємо статистичні показники
    return filtered_df.groupby(["PROVINCE_ID", "Year"])["VHI"].agg(["min", "max", "mean", "median"]).reset_index()

# Виклик функції для областей з ID 5 і 10 у 2000 та 2020 роках
stats = get_vhi_statistics(combined_data, province_ids=[5, 10], years=[2000, 2020])
print(stats)


Empty DataFrame
Columns: [PROVINCE_ID, Year, min, max, mean, median]
Index: []


3. Отримати ряд VHI за вказаний діапазон років для вказаних областей

In [7]:
def get_vhi_by_year_range(df, province_ids, start_year, end_year):

    result = df[(df["PROVINCE_ID"].isin(province_ids)) & (df["Year"].between(start_year, end_year))]
    
    if result.empty:
        print("Дані для вказаного діапазону відсутні!")
    
    return result[["Year", "Week", "PROVINCE_ID", "VHI"]]

# Використання:
province_ids = [3, 7, 12]  # Введи потрібні області
start_year = 2010  # Початковий рік
end_year = 2020  # Кінцевий рік
vhi_range_data = get_vhi_by_year_range(combined_data, province_ids, start_year, end_year)
print(vhi_range_data)


Дані для вказаного діапазону відсутні!
Empty DataFrame
Columns: [Year, Week, PROVINCE_ID, VHI]
Index: []


o Для всього набору даних виявити роки, протягом яких екстремальні
посухи торкнулися більше вказаного відсотка областей по Україні (20%
областей - 5 областей з 25). Повернути роки, назви областей з
екстремальними посухами та значення VHI;

4. Виявити роки, коли посуха торкнулася >20% областей (VHI < 15)

In [8]:
def find_drought_years(df, threshold=15, affected_percentage=0.2):

    province_count = df["PROVINCE_ID"].nunique()  # Загальна кількість областей
    min_affected = int(province_count * affected_percentage)  # Мінімальна кількість уражених областей

    drought_data = df[df["VHI"] < threshold].groupby(["Year"])["PROVINCE_ID"].nunique().reset_index()
    drought_years = drought_data[drought_data["PROVINCE_ID"] >= min_affected]

    if drought_years.empty:
        print("Не знайдено років, коли посуха торкнулася 20% областей.")
        return None

    # Отримуємо детальні дані про області, які потрапили під посуху
    drought_details = df[(df["Year"].isin(drought_years["Year"])) & (df["VHI"] < threshold)]
    
    return drought_details[["Year", "PROVINCE_ID", "VHI"]]

# Використання:
drought_info = find_drought_years(combined_data)
print(drought_info)


Не знайдено років, коли посуха торкнулася 20% областей.
None


In [9]:
#Чому дорівнює: 
a = [n**2+1 for n in range(5)]
print(a)


[1, 2, 5, 10, 17]


In [10]:
print(dir(__builtins__))

['ArithmeticError', 'AssertionError', 'AttributeError', 'BaseException', 'BaseExceptionGroup', 'BlockingIOError', 'BrokenPipeError', 'BufferError', 'BytesWarning', 'ChildProcessError', 'ConnectionAbortedError', 'ConnectionError', 'ConnectionRefusedError', 'ConnectionResetError', 'DeprecationWarning', 'EOFError', 'Ellipsis', 'EncodingWarning', 'EnvironmentError', 'Exception', 'ExceptionGroup', 'False', 'FileExistsError', 'FileNotFoundError', 'FloatingPointError', 'FutureWarning', 'GeneratorExit', 'IOError', 'ImportError', 'ImportWarning', 'IndentationError', 'IndexError', 'InterruptedError', 'IsADirectoryError', 'KeyError', 'KeyboardInterrupt', 'LookupError', 'MemoryError', 'ModuleNotFoundError', 'NameError', 'None', 'NotADirectoryError', 'NotImplemented', 'NotImplementedError', 'OSError', 'OverflowError', 'PendingDeprecationWarning', 'PermissionError', 'ProcessLookupError', 'RecursionError', 'ReferenceError', 'ResourceWarning', 'RuntimeError', 'RuntimeWarning', 'StopAsyncIteration', 'S

In [11]:
dir( __builtins__ )

['ArithmeticError',
 'AssertionError',
 'AttributeError',
 'BaseException',
 'BaseExceptionGroup',
 'BlockingIOError',
 'BrokenPipeError',
 'BufferError',
 'BytesWarning',
 'ChildProcessError',
 'ConnectionAbortedError',
 'ConnectionError',
 'ConnectionRefusedError',
 'ConnectionResetError',
 'DeprecationWarning',
 'EOFError',
 'Ellipsis',
 'EncodingWarning',
 'EnvironmentError',
 'Exception',
 'ExceptionGroup',
 'False',
 'FileExistsError',
 'FileNotFoundError',
 'FloatingPointError',
 'FutureWarning',
 'GeneratorExit',
 'IOError',
 'ImportError',
 'ImportWarning',
 'IndentationError',
 'IndexError',
 'InterruptedError',
 'IsADirectoryError',
 'KeyError',
 'KeyboardInterrupt',
 'LookupError',
 'MemoryError',
 'ModuleNotFoundError',
 'NameError',
 'None',
 'NotADirectoryError',
 'NotImplemented',
 'NotImplementedError',
 'OSError',
 'OverflowError',
 'PendingDeprecationWarning',
 'PermissionError',
 'ProcessLookupError',
 'RecursionError',
 'ReferenceError',
 'ResourceWarning',
 'Runti

In [12]:
print dir( __builtins__ )

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(...)? (2517868533.py, line 1)

In [ ]:
planets = ['Mercury', 'Venus', 'Earth', 'Mars', 'Jupiter', 'Saturn', 'Uranus', 'Neptune'][3:-2]
print(planets)

['Mars', 'Jupiter', 'Saturn']


в1

In [ ]:
def least_difference(a, b, c):
    diff1 = abs(a - b)
    diff2 = abs(b - c)
    diff3 = abs(a - c)
    return min(diff1, diff2, diff3)
print(least_difference(10, 5, 8)) 


2


In [ ]:
def least_difference(a, b, c):
    diff1 = a - b
    diff2 = b - c
    diff3 = a - c
    return min(diff1, diff2, diff3)
print(least_difference(10, 5, 8)) 

-3


In [ ]:
def least_difference(a, b, c):
    diff1 = min(a - b)
    diff2 = min(b - c)
    diff3 = min(a - c)
    return min(diff1, diff2, diff3)
print(least_difference(10, 5, 8)) 

TypeError: 'int' object is not iterable

In [ ]:
def least_difference(a, b, c):
    diff1 = a - b
    diff2 = b - c
    diff3 = a - c
    return min(diff1, diff2, diff3)
print(least_difference(10, 5, 8)) 

-3


In [ ]:
True and ( False or ( True and False or True))


True